<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/VALL_E_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/Plachtaa/VALL-E-X.git
%cd VALL-E-X

fatal: destination path 'VALL-E-X' already exists and is not an empty directory.
/content/VALL-E-X


In [4]:
!pwd

/content/VALL-E-X


In [5]:
!pip install -r requirements.txt

  Using cached websockets-11.0.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (129 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yt-dlp 2023.12.30 requires websockets>=12.0, but you have websockets 11.0.3 which is incompatible.
  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 11.0.3
    Uninstalling websockets-11.0.3:
      Successfully uninstalled websockets-11.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [6]:
from utils.generation import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

In [8]:
preload_models()

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
text_prompt = """
    This is a pen.
"""
audio_array = generate_audio(text_prompt)

write_wav("/content/english.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE)

VALL-E EOS [0 -> 62]


In [34]:
from utils.prompt_making import make_prompt

# 先ほど生成した音声を使う
make_prompt(name="sample", audio_prompt_path="/content/input_clip.wav",
                transcript="警察と検察は証拠の確保に失敗したのです。")

text_prompt = """
こんにちは、堺雅人です。
"""
audio_array = generate_audio(text_prompt, prompt="sample")

write_wav("/content/sample2.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE)

VALL-E EOS [304 -> 504]


In [16]:
!pip install yt-dlp
!pip install imageio
!pip install pydub

In [19]:
from yt_dlp import YoutubeDL

from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import IPython.display as ipd

In [25]:
video_url = 'https://www.youtube.com/watch?v=pICoRxvNiFk' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  0#@param {type:"integer"}
end_sec =  4#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [29]:
!mkdir -p videos audios

download_resolution = 360
full_video_path = '/content/full_video.mp4'
input_clip_path = '/content/input_clip.mp4'
input_audio_path = '/content/input_clip.wav'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=pICoRxvNiFk
[youtube] pICoRxvNiFk: Downloading webpage
[youtube] pICoRxvNiFk: Downloading ios player API JSON
[youtube] pICoRxvNiFk: Downloading android player API JSON
[youtube] pICoRxvNiFk: Downloading m3u8 information
[info] pICoRxvNiFk: Downloading 1 format(s): 18
[download] Destination: /content/full_video.mp4
[download] 100% of    9.44MiB in 00:00:00 at 16.81MiB/s  


In [30]:
# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

Moviepy - Building video /content/input_clip.mp4.
MoviePy - Writing audio in input_clipTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/input_clip.mp4



Moviepy - Done !
Moviepy - video ready /content/input_clip.mp4


In [31]:
# 音声抽出
clip = VideoFileClip(input_clip_path)
clip.audio.write_audiofile(input_audio_path, codec='pcm_s16le')

ipd.Audio(input_audio_path, rate=44100)


MoviePy - Writing audio in /content/input_clip.wav


MoviePy - Done.


In [2]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b

SyntaxError: incomplete input (<ipython-input-2-f752bb8b4e91>, line 33)